## 코드 설명
* 이 코드는 로그인 후에 제품, 가격의 크롤링을 하는 코드입니다. 


* 필요한 부분을 수정하여 커머스 홈페이지의 상품과 가격을 수집하는 용도로 사용할 수 있습니다.


* name과 css_selector로 찾은 부분은 실제 html을 직접 확인하시고 적절히 바꿔주셔야 합니다. 수정이 쉽도록 최대한 상세한 설명을 덧붙여 보았습니다. 


* 다만, 상업적 용도로 주기적으로 데이터를 수집하는 행위는 경우에 따라 법적 제재가 있을 수 있으니 유의하세요! 

## 기본세팅

In [ ]:
# chromedriver 다운로드
# !pip install chromedriver-autoinstaller
# 셀레니움 설치 
# !pip install selenium

In [ ]:
import chromedriver_autoinstaller  # 크롬드라이버 자동 설치 및 업데이트

import requests  # HTTP 요청을 보내 웹페이지의 HTML 가져오기

from bs4 import BeautifulSoup as bs  # HTML을 파싱하여 원하는 데이터 추출

import time  # 코드 실행 간 대기 시간 설정 (예: time.sleep)
from random import randint  # 랜덤한 정수 생성 (예: 요청 간격 조정)
import pandas as pd  # 데이터 분석 및 엑셀, CSV 저장

from selenium import webdriver  # 웹 브라우저 자동화
from selenium.webdriver.common.by import By  # HTML 요소 찾기 (예: By.CSS_SELECTOR)
from selenium.webdriver.common.action_chains import ActionChains  # 마우스/키보드 액션 자동화
from selenium.webdriver.support.ui import WebDriverWait  # 특정 요소가 나타날 때까지 대기
from selenium.webdriver.support import expected_conditions as EC  # 특정 조건 충족 여부 확인
from selenium.webdriver.common.keys import Keys  # 키보드 입력 자동화 (예: 엔터 키 입력)

# 자체 정의 함수 
from action_on_page.scroll import scroll_down

## 로그인

In [ ]:
# 로그인 페이지 설정 
login_url = '서비스의 로그인 페이지 url'

# 1. 크롬 드라이버 실행 
driver = webdriver.Chrome()

# 2. 로그인 페이지 접속 
driver.get(login_url)

# 3. 로그인 정보 입력 
user_id = 'id' # 아이디 입력
user_pw = 'pw' # 패스워드 입력

# 4. ID 입력 필드 찾고 값 입력 
id_input = driver.find_element(By.NAME, "loginId")
id_input.send_keys(user_id)

# 5. 비밀번호 입력 필드 찾고 값 입력
pw_input = driver.find_element(By.NAME, "loginPwd")  
pw_input.send_keys(user_pw)  

# 6. 로그인 버튼 클릭
login_button = driver.find_element(By.CSS_SELECTOR, "#formLogin > fieldset > div.submit > button")
login_button.click()

# 7. 로그인 완료 후 잠시 대기 후 성공여부 확인 (페이지 로딩 고려)
time.sleep(3)

if "로그아웃" in driver.page_source:  
    print("로그인 성공!")
else:  
    print("로그인 실패!")

## 특정 카테고리 스크래핑

In [ ]:
#토너크림에센스 카테고리
category_url = '특정 카테고리 url 입력'
driver.get(category_url) 

In [ ]:
# 반복되지 않는 부분 미리 정의하기 
item_name = []
item_price = []
item_img = []

In [ ]:
while True:  # 페이지가 끝날 때까지 계속 실행
    print("현재 페이지에서 데이터 수집 중...")

    # 스크롤을 내려서 모든 상품 로드
    scroll_down(driver)

    # 페이지의 정보를 가져오기
    soup = bs(driver.page_source)

    # 해당 정보가 있는 div 찾기
    item_name_lst = soup.find_all('div', class_='item_tit_box')  
    item_price_lst = soup.find_all('div', class_='item_money_box')  
    item_img_lst = soup.find_all('div', class_='item_photo_box')
    
    # strong class 찾아서 저장하기
    item_names = [item.find('strong', class_='item_name').text.strip() for item in item_name_lst]
    item_prices = [item.find('strong', class_='item_price').text.strip() for item in item_price_lst]
    item_imgs = [img.find('img')['src'] for img in item_img_lst]
  
    # 리스트에 데이터 추가
    item_name.extend(item_names)
    item_price.extend(item_prices)
    item_img.extend(item_imgs)
    
    # 현재 페이지 번호 찾기
    try:
        current_page = driver.find_element(By.CSS_SELECTOR, ".pagination li.on span").text.strip()
        print(f"현재 페이지: {current_page}")
        
        # 다음 페이지 버튼 찾기
        page_buttons = driver.find_elements(By.CSS_SELECTOR, ".pagination li a")  # 페이지 번호 버튼 목록 가져오기

        next_page_found = False
        for button in page_buttons:
            if button.text.strip() == str(int(current_page) + 1):  # 현재 페이지 + 1인 버튼 찾기
                button.click()
                time.sleep(3)  # 페이지 로딩 대기
                next_page_found = True
                break
        
        # 다음 페이지 버튼이 없으면 종료
        if not next_page_found:
            print("마지막 페이지입니다. 크롤링 종료.")
            break  

    except Exception as e:
        print("페이지 이동 중 오류 발생:", e)
        break  # 오류 발생 시 크롤링 종료


In [ ]:
#데이터프레임: 제품 이름, 가격
df = pd.DataFrame()
df['item_name'] = item_name 
df['item_price'] = item_price
df['item_img_src'] = item_img

df

In [ ]:
## 엑셀 형태로 내보내기
df.to_excel('./250101_item_list.xlsx')

## Next Level
* **env. 파일 만들기** 

추후 코드에서 수정할 부분은 개인정보를 따로 빼놓은 .env 파일에 id와 pw를 저장하는 것입니다. 사용할 때는 일회용 코드라서 따로 .env 코드를 만들지는 않았으나 .env를 만들어 id와 pw를 저장하면 github에 올리실 때 gitignore 기능을 사용하여 보안에 유의해야 하는 정보를 숨기고 코드를 공유할 수 있습니다.


* **스크래핑 자체정의 함수 구현하기**

스크롤 기능만 자체정의 함수로 구현해 두었는데, 카테고리별로 반복적으로 스크래핑을 할 수 있으니 이 부분도 자체 정의 함수를 구현해두어야겠다는 생각이 들었습니다.